## Interpolation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.interpolate import interpn
from astropy import constants as c

Note: this required [this fork of pyPLUTO](https://gitlab.mpcdf.mpg.de/sdoetsch/pypluto.git) to work, because of changes in the standard library, see [this fix](https://gitlab.mpcdf.mpg.de/sdoetsch/pypluto/-/commit/7b8243c6073785d1486f1bd495a98082f09f95f3)

In [ ]:
import pyPLUTO as pp
au = c.au.cgs.value

In [ ]:
pp.pload.pload(55, w_dir='/Users/birnstiel/Desktop/PLUTO_Planet/data/', datatype='flt')

In [ ]:
data = pp.pload.pload(55, datatype='flt', w_dir='/Users/birnstiel/Desktop/PLUTO_Planet/data/')

UNIT_DENSITY  = 3.604e-13 # (gr/cm^3)
UNIT_VELOCITY = 6.679e+05 # (cm/s)
UNIT_LENGTH   = 2.992e+14 # (cm)

r = data.x1 * UNIT_LENGTH
ri = data.x1r * UNIT_LENGTH
th = data.x2
thi = data.x2r
ph = data.x3
phi = data.x3r
rho = data.rho.copy() * UNIT_DENSITY

## Inspect the data

Here are the original data dimensions

In [ ]:
r.shape, th.shape, ph.shape

In [ ]:
rho.shape

make a 2D vertical slice

In [ ]:
rri, tti = np.meshgrid(ri, thi, indexing='ij')

xxi = rri * np.sin(tti)
zzi = rri * np.cos(tti)

i_phi = 151

f, ax = plt.subplots(dpi=200)
vmax = 10.**np.ceil(np.log10(rho.max()))
cc =ax.pcolormesh(xxi / au, zzi / au, rho[:, :, i_phi], norm=LogNorm(vmin=1e-3 * vmax, vmax=vmax), shading='flat')
ax.set_aspect('equal')
pos = ax.get_position()
cax = f.add_axes([pos.x1, pos.y0, pos.height / 30, pos.height])
cb = plt.colorbar(cc, cax=cax)
cb.set_label('$\\rho_{gas}$')

## Prepare for interpolation

also a bit tricky: we define the density only grid centers, so there is a gap in phi direction between the last and the first grid center as the interpolation will not know about the periodic direction. We close this here by adding another point beyond $2\pi$ which is a copy of the first point near $\phi = 0$.

In [ ]:
ph_mod = np.hstack((ph, 2 * np.pi + ph[0]))
rho_mod = np.concatenate((rho, rho[:, :, 0:1]), axis=2)

We create a cartesian slice, here in the mid-plane `(x, y)` while we call the height `z`

In [ ]:
dx = 0.14

x = np.arange(-35, 35, dx) * au
y = np.arange(-36, 36, dx) * au
z = np.arange(-5, 5, dx) * au

X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

We translate the coordinates of that slice to spherical coordinates.

Note that phi goes from 0 to $2 \pi$ in the original data, but the output of `np.arctan2` has negative angles and needs to be shifted

In [ ]:
R = np.sqrt(X**2 + Y**2 + Z**2)
T = np.pi/2 - np.arctan2(Z, np.sqrt(X**2 + Y**2))
P = (np.arctan2(Y, X) + 2 * np.pi) % (2 * np.pi)

We create a array of new points, shape is (N, 3)

In [ ]:
points = np.array([R.ravel(), T.ravel(), P.ravel()]).T

We call the interpolation. There's values outside the box, so we assign them the value `0.0`.

In [ ]:
interp = interpn((r, th, ph_mod), rho_mod, points, fill_value=0.0, bounds_error=False)

The result is again one dimensional (one value per new point), so we need to reshape it to match the shape of the slice. After this, it is again 3-dimensional, since our slice has 3 dimensions, but the z-dimension is just one value here.

In [ ]:
interp = interp.reshape(X.shape)

Plot it

In [ ]:
X.shape

In [ ]:
f, axs = plt.subplots(1, 2, dpi=200, gridspec_kw={'width_ratios':[4, 1]}, figsize=(10, 2))

vmax = 10.**np.ceil(np.log10(interp.max()))

iy = len(y) // 2
iz = len(z) // 2

ax = axs[0]
cc1 = ax.pcolormesh(X[:, iy, :] / au, Z[:, iy, :] / au, interp[:, iy, :], norm=LogNorm(vmin=1e-3 * vmax, vmax=vmax), shading='auto', rasterized=True)

ax = axs[1]
cc2 = ax.pcolormesh(X[:, :, iz] / au, Y[:, :, iz] / au, interp[:, :, iz], norm=LogNorm(vmin=1e-3 * vmax, vmax=vmax), shading='auto', rasterized=True)


for ax, cc in zip(axs, [cc1, cc2]):
    ax.set_aspect('equal')
    pos = ax.get_position()
    cax = f.add_axes([pos.x0, pos.y1, pos.width, pos.height / 20])
    cb = plt.colorbar(cc, cax=cax, orientation='horizontal')
    cb.set_label('$\\rho_{gas}$')
    cax.xaxis.set_label_position('top')
    cax.xaxis.set_ticks_position('top')
    
f.savefig('plot.pdf', transparent=True, bbox_inches='tight')

## Export data

store the data in a simple format

In [ ]:
np.savez('pluto_data.npz', x=x, y=y, z=z, rho=interp)

this can be loaded with:

In [ ]:
with np.load('pluto_data.npz') as f:
    x = f['x']
    y = f['y']
    z = f['z']
    rho = f['rho']